# SE-Snek: SE-Sync Python bindings 🐍

This notebook demonstrates the use of SE-Sync's Python bindings

In [ ]:
import sys
import numpy as np
from matplotlib import pyplot as plt

Import the sesync Python library built using pybind

In [ ]:
sesync_lib_path = "../build/lib"  # Relative path to libraries built by the SE-Sync CMake project
sys.path.insert(0, sesync_lib_path)  # Add this directory to Python interpreter path

In [ ]:
import sesync

Load in some data from a .g2o file

In [ ]:
filename = "../../data/manhattan.g2o"

measurements, num_poses = sesync.read_g2o_file(filename)

print("Loaded %d measurements between %d poses from file %s" % (len(measurements), num_poses, filename))

**Run SE-Sync!**

Set SE-Sync options

In [ ]:
opts = sesync.SESyncOpts()
opts.num_threads = 4
opts.verbose=True

opts.r0 = 2
opts.formulation = sesync.Formulation.Explicit
opts.initialization = sesync.Initialization.Random

In [ ]:
result = sesync.SESync(measurements, opts)

Print out some interesting stats

In [ ]:
print("SESync return status: %s\n\n\
Initial relaxation rank: %d\n\
Final relaxation rank: %d\n\
Value of dual SDP solution F(Y):  %g\n\
Norm of Riemannian gradient gradF(Y): %g\n\
Value of primal SDP solution tr(Lambda): %g\n\
Minimum eigenvalue of certificate matrix: %g\n\
SDP duality gap: %g\n\n\
Value of rounded pose estimates F(x): %g\n\
Suboptimality bound of recovered pose estimates: %g\n\n\
Total elapsed computation time: %g seconds " % (result.status, opts.r0, result.Yopt.shape[0], result.SDPval, result.gradnorm, 
         result.trLambda, result.lambda_min, result.duality_gap, 
        result.Fxhat, result.suboptimality_bound, result.total_computation_time))

**Plot some interesting data about the solution process**

In [ ]:
# Plot sequence of Riemannian gradient norms
grad_norms = np.concatenate(result.gradient_norms)
plt.semilogy(grad_norms)
plt.title("Norm of Riemannian gradient")
plt.xlabel("Iteration")
plt.show()

In [ ]:
# Plot sequence of objective values
fvals = np.concatenate(result.function_values)
plt.semilogy(fvals)
plt.title("Objective value")
plt.xlabel("Iteration")
plt.show()

**Plot estimated SLAM solution**

In [ ]:
# Extract translational states from solution xhat
xhat = result.xhat
R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + 2])
t = np.matmul(R0inv, xhat[:, 0:num_poses])

# Plot the position of each estimated pose, together with the edges between them
for k in range(0, len(measurements)):
    plt.plot(t[0, [measurements[k].i, measurements[k].j]], t[1, [measurements[k].i, measurements[k].j]], '.-b')

plt.title("Estimated SLAM solution")

plt.gca().set_aspect('equal')
plt.show()